In [1]:
import matplotlib.pyplot as plt
import time
import cupy as cp #Use if GPU is avilable other wise
import numpy as np

'''Declearning Constand and Inputs =================='''
t1= time.time()
# Constants (all MKS, except energy which is in eV)
hbar = 1.06e-34
q = 1.6e-19
IE = (q*q) / (2 * cp.pi * hbar)
Ef = 2.25
kT = 0.025
Ecc = 3.02

#Effective mass in lattice for insulator and ferromagnet
mc = 0.18*9.1e-31
mf = 0.8*9.1e-31

# Inpupt a =size of lattice
a = 0.25e-10

#t0, tm are values of t in channel and magnet repectivily (NOTE:for now we have asumme energy Ec is same in both)
t0 = (hbar**2) / (2 * mc * (a**2) * q)
tm = (hbar**2) / (2 * mf * (a**2) * q)

#Lattic point in Source, channel and Drain
NS = 2
NC = 38
ND = 2
No = NS + NC + ND

#Loading Relevent arrays (Hamaltonia, Eigen Basis and Unitary Transformation)
Unit_R = cp.genfromtxt("Unit_R.csv", delimiter=",",dtype=complex)
Unit_L = cp.genfromtxt("Unit_L.csv", delimiter=",",dtype=complex)
T_trans = cp.genfromtxt("T_trans.csv", delimiter=",")
TR = cp.genfromtxt("TR.csv", delimiter=",",dtype=complex)
TL = cp.genfromtxt("TL.csv", delimiter=",")
Eig_L = cp.genfromtxt("Eig_L.csv", delimiter=",")
Eig_R = cp.genfromtxt("Eig_R.csv", delimiter=",")

Np = 986  #take half of legth of Eig_L


In [ ]:
#Defining potentail profile
UB = cp.zeros(No)
UB[NS:NS+NC]=Ecc

# Bias
V_range = cp.linspace(0, 0, 1) # Range of bias voltages to calculate
V_range = [0.02]
I_array = [] # Array to store the currents for each bias voltage

NE=20

t1=time.time()

sx = cp.array([[0,1],[1,0]])
smx = cp.kron(sx,cp.eye(Np))

for V in V_range:
    mu1 = Ef + (V/2)
    mu2 = Ef - (V/2)
    #Applying voltage and potentail texture
    UX = V* cp.concatenate((0.5 * cp.ones(2*NS), cp.linspace(0.5, -0.5, 2*NC), -0.5 * cp.ones(2*ND)))
    U1 = UX[0:2*No:2]
    U1=U1+UB
    # Transmission
    TM = cp.zeros(NE)
    # Energy grid for Green's function method
    E = cp.linspace(mu2-10*kT, mu1+10*kT, NE)
    zplus = 1j * 1e-12
    dE = E[1] - E[0]
    f1 = 1 / (1 + cp.exp((E - mu1) / kT))
    f2 = 1 / (1 + cp.exp((E - mu2) / kT))
    Is=cp.zeros(NE)
    Iops = cp.zeros((2*Np,2*Np),dtype=complex)
    I=0
    Itx=0
    for k in range(NE):
      print(k,end="")
      '''Getting surface green function for both contacts'''
      #Right contact
      #Adjust eignvalues according to mass of contact
      ck = 1 - ((E[k] + zplus-U1[0] -Eig_L) / (2*tm))
      ka = cp.arccos(ck)
      Green = -tm * cp.exp(1j*ka)
      sigm1 = Unit_L@cp.diag(Green)@Unit_L.T
      
      #Left contact
      ck = 1 - ((E[k] + zplus -U1[No-1] -Eig_R)/(2*tm))
      ka = cp.arccos(ck)
      Green = -tm * cp.exp(1j*ka)
      sigm2 = Unit_R@cp.diag(Green)@cp.conj(Unit_R.T)
      #Calcuating sigmaprint(Eig_L[2*Np-1])
      '''NOTE: Please Run the Next script first to intilize these inverse function''' 
      [G11,GN1]=Inv_surf_D(T_trans,TL,TR,E[k],U1,0,sigm1,sigm2)
      [G11,GN2]=Inv_surf_D(T_trans,TL,TR,E[k],U1,1,sigm1,sigm2)
      [G11,G12]=Inv_surf(T_trans,TL,TR,E[k],U1,2,sigm1,sigm2)

      #Calculating gama1 and gama2
      gama1 = 1j*(sigm1-cp.conj(sigm1.T)) 
      gama2 = 1j*(sigm2-cp.conj(sigm2.T))

      #Calculating [1,2] and [2,1] enteries of A1 and A2
      A112= G11@gama1@(cp.conj(G12).T)
      A121 = G12@gama1@(cp.conj(G11).T)
      A212 = GN1@gama2@(cp.conj(GN2).T)
      A221= GN2@gama2@(cp.conj(GN1).T) 
      
      #GN1 is GN[0,1] and GN2 is GN][1,0] 
      GNN1 = A112*f1[k]+A212*f2[k]
      GNN2 = A121*f1[k]+A221*f2[k]
      Iop = 1j*tm*(GNN1-GNN2)
      Iops+=Iop*dE*IE    #Saving current profile
      Is[k]=IE*cp.real(cp.trace(Iop))
      #Eigenvalue wise spin current and charge current
      
      I += Is[k]*dE
  
t2=time.time()
#Saving current profile
cp.savetxt("Iop.csv", Iops, delimiter=",")

print("\n I")
print(I)
#print(Isz,'/',Isx,'/',Isy)

0123456

In [7]:
print(cp.real(cp.trace(Iops)))
print(cp.real(cp.trace(Iops[:Np,:Np]-Iops[Np:,Np:])))

3.1616517075847306e-06
1.8640802799839435e-06


In [2]:
### Contain function that can calcuate specific section of inverse

def Inv_surf(T_trans,T_L,T_R,E,V,p,sigm1,sigm2):
  '''This matrix calcuate the first row of a Green function, T is the traverse hamatonain(without effect of magnetization and effective mass is taken that of a channel), E is Energy, V is votlage profile p is element of raw you want to calcuate
  Altough Simg1 and simg2 are sigma's of respective matrix but modification due to hamatonain due to magnetic texture can be added with these'''
  
  #B is will (1,1) matrix and Xm will be (1,p) matrix 
  B=cp.zeros((2*Np,2*Np),dtype=complex)
  Xm=cp.zeros((2*Np,2*Np),dtype=complex)
  #Adding effect of tight binding in long direction
  Ez = (E+zplus)*cp.eye(2*Np)
  Mcon = tm/t0 #To convert system with effective mass of mc to mf
  Mjuc = (tm+t0)/(2*t0) #To convert system with effective mass of mc to mf+mc/2
  #Calculating surface of first 2x2 matrix
  A =Ez-T_R - sigm2 -V[No-1]*cp.eye(2*Np)
  B = cp.linalg.inv(A)
  if p==No:
      Xm = B
  for i in range(No-2):
    if i==0:
      A =Ez-((T_trans)*Mjuc) -V[No-2-i]*cp.eye(2*Np)
      B = A - (tm**2)*B
    elif i == No-3:
      A =Ez-((T_trans)*Mjuc) - V[No-2-i]*cp.eye(2*Np)
      B = A - (t0**2)*B
    else:
      A =Ez-(T_trans) - V[No-2-i]*cp.eye(2*Np)
      B = A - (t0**2)*B
    B = cp.linalg.inv(B)
    if i==No-1-p:
      Xm = B
    if i>=No-p:
      if i==0:
        Xm= -1*(tm)*Xm@B
      else:
        Xm = -1*(t0)*Xm@B

  A =Ez-T_L - sigm1 - V[0]*cp.eye(2*Np)
  B = (A) - (tm**2)*B
  B = cp.linalg.inv(B)
  Xm = -1*(tm)*Xm@B
  return [B,Xm]

  '''---------------------------------------------------------------------------'''


def Inv_surf_D(T_trans,T_L,T_R,E,V,p,sigm1,sigm2):
  '''This matrix calcuate the first row of a Green function, T is the traverse hamatonain(without effect of magnetization and effective mass is taken that of a channel), E is Energy, V is votlage profile p is element of raw you want to calcuate
  Altough Simg1 and simg2 are sigma's of respective matrix but modification due to hamatonain due to magnetic texture can be added with these'''
  
  #B is will (1,1) matrix and Xm will be (1,p) matrix 
  B=cp.zeros((2*Np,2*Np),dtype=complex)
  Xm=cp.zeros((2*Np,2*Np),dtype=complex)
  #Adding effect of tight binding in long direction
  Ez = (E+zplus)*cp.eye(2*Np)
  Mcon = tm/t0 #To convert system with effective mass of mc to mf
  Mjuc = (tm+t0)/(2*t0) #To convert system with effective mass of mc to mf+mc/2
  #Calculating surface of first 2x2 matrix
  A =Ez-TL - sigm1 -V[0]*cp.eye(2*Np)
  B = cp.linalg.inv(A)
  if p==0:
      Xm = B
  for i in range(1,No-1):
    if i==1:
      A =Ez-((T_trans)*(Mjuc)) -V[i]*cp.eye(2*Np)
      B = A - (tm**2)*B
    elif i == No-2:
      A =Ez-((T_trans)*(Mjuc)) - V[i]*cp.eye(2*Np)
      B = A - (t0**2)*B
    else:
      A =Ez-(T_trans) - V[i]*cp.eye(2*Np)
      B = A - (t0**2)*B
    B = cp.linalg.inv(B)

    if i==p:
      Xm=B
    if i>=p+1:
      if i==1:
        Xm=-tm*Xm@B
      else:  
        Xm=-t0*Xm@B

  A =Ez-TR - sigm2 - V[No-1]*cp.eye(2*Np)
  B = (A) - (tm**2)*B
  B = cp.linalg.inv(B)
  Xm = -1*(tm)*Xm@B
  return [B,Xm]


